<a href="https://colab.research.google.com/github/deanhoperobertson/Named-Enitty-Recognition/blob/master/BiLSTM-CRF%20Model%20(%2BUncased%20Embeddings).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embeddings Bi-LSTM-CRF Model

In [0]:
from google.colab import files
uploaded = files.upload()

Saving prepro.py to prepro (1).py


In [0]:
!sudo pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-x_wgilb0
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-x_wgilb0
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=1ec82ffd18d63e63174581e0194fba24bdc83d19c0abbc9f1aa0b9030ced90ba
  Stored in directory: /tmp/pip-ephem-wheel-cache-y691km61/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
!pip install sklearn_crfsuite

In [0]:
import pandas as pd
import numpy as np
import urllib.request
from sklearn.model_selection import train_test_split

#cusotm packages
from prepro import readfile, readstring, partial_tags

#keras and tensorflow packages
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy


#evaluation
from sklearn_crfsuite.metrics import flat_classification_report,flat_f1_score,flat_precision_score

In [0]:
#import data from my github repo
train_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/train.txt"
test_url = "https://raw.githubusercontent.com/deanhoperobertson/Masters-/master/Thesis/Code/Data/test.txt"
train = urllib.request.urlopen(train_url).read()
test = urllib.request.urlopen(test_url).read()
train = train.decode('utf-8')
test = test.decode('utf-8')

#preproces the txt file
train = readstring(train)
test = readstring(test)

#create corpus
corpus = train.copy()
corpus.extend(test)

In [0]:
words = []
tags = []
for sentence in corpus:
    for word in sentence:
        words.append(word[0])
        tags.append(word[1])
        
tags = partial_tags(tags)

setlabel=list(set(tags))
setlabel.remove('O')
        
words= [w.lower() for w in words]

n_words = len(set(words))
print("Number of words in the dataset: ", n_words)
n_tags = len(list(set(tags)))
print("Number of Labels: ", n_tags)

Number of words in the dataset:  24277
Number of Labels:  5


In [0]:
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(list(set(tags)))}
tag2idx["PAD"] = 0
# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}
print("The labels LOC (location) is identified by the index: {}".format(tag2idx["ORG"]))

The labels LOC (location) is identified by the index: 4


In [0]:
word2idx = {w: i + 2 for i, w in enumerate(list(set(words)))}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding
# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}
print("The word 'rejects' is identified by the index: {}".format(word2idx["rejects"]))

The word 'rejects' is identified by the index: 7463


In [0]:
#Find the maxium length of the all the sentences in the corpus
length = []
for sentence in corpus:
  length.append(len(sentence))

MAX_LEN= max(length)
print("The maxium length of sentence is:",max(length))

The maxium length of sentence is: 124


In [0]:
# Convert each sentence from list of Token to list of word_index
X = [[word2idx[w[0].lower()] for w in s] for s in train]

# Padding each sentence to have the same lenght
X = pad_sequences(maxlen=MAX_LEN, sequences=X, padding="post", value=word2idx["PAD"])
X[0]

array([ 1911,  7463,  8712, 23509,  6525, 11456, 12008, 18258, 19676,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0], dtype=int32)

In [0]:
def get_labeler(word):
  if word =="O":
    return word
  elif "MISC" in word:
    return word[-4:]
  else:
    return word[-3:]

# Convert Tag/Label to tag_index
y = [[tag2idx[get_labeler((w[1]))] for w in s] for s in train]

# Padding each sentence to have the same lenght
y = pad_sequences(maxlen=MAX_LEN, sequences=y, padding="post", value=tag2idx["PAD"])
y[0]

array([4, 3, 1, 3, 3, 3, 1, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
# One-Hot encode categorical labels
y = [to_categorical(i, num_classes=n_tags+1) for i in y]

## Adding Embeddings

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

#root_path = "/content/drive/My Drive/glove.6B.50d.txt"
#root_path = "/content/drive/My Drive/glove.6B.100d.txt"
#root_path = "/content/drive/My Drive/glove.6B.200d.txt"
root_path = "/content/drive/My Drive/glove.6B.300d.txt"
#root_path = "/content/drive/My Drive/wiki-news-300d-1M.vec"

embeddings_index={}
f = open(root_path, encoding = "utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
#create emedding matrix

EMBEDDING=300
embedding_matrix = np.zeros((len(word2idx) + 1, EMBEDDING))
for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
# Model definition

input = Input(shape=(MAX_LEN,))

model_2 = Embedding(len(word2idx)+1,output_dim=EMBEDDING,
                  weights=[embedding_matrix],input_length=MAX_LEN,
                  trainable=False,mask_zero=True)(input)

model_2 = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(model_2)

model_2 = Bidirectional(LSTM(units=64, return_sequences=True,
                           recurrent_dropout=0.1))(model_2)

model_2 = TimeDistributed(Dense(10, activation="relu"))(model_2)
crf = CRF(n_tags+1)  # CRF layer
out = crf(model_2)  # output
model_2 = Model(input, out)

In [0]:
model_2.compile(optimizer='adam', loss=crf_loss,metrics=[crf_viterbi_accuracy])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 124)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 124, 300)          7284000   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 124, 128)          186880    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 124, 128)          98816     
_________________________________________________________________
time_distributed_6 (TimeDist (None, 124, 10)           1290      
_________________________________________________________________
crf_6 (CRF)                  (None, 124, 6)            114       
Total params: 7,571,100
Trainable params: 287,100
Non-trainable params: 7,284,000
____________________________________________________________

In [0]:
%%time
BATCH_SIZE = 200
EPOCHS=15

history = model_2.fit(X, np.array(y), batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

Train on 12636 samples, validate on 1405 samples
Epoch 1/15
12636/12636 [==============================] - 86s 7ms/step - loss: 33.1890 - crf_viterbi_accuracy: 0.7903 - val_loss: 27.7826 - val_crf_viterbi_accuracy: 0.8842
Epoch 2/15
12636/12636 [==============================] - 78s 6ms/step - loss: 32.6544 - crf_viterbi_accuracy: 0.9018 - val_loss: 27.5091 - val_crf_viterbi_accuracy: 0.9286
Epoch 3/15
12636/12636 [==============================] - 78s 6ms/step - loss: 32.4781 - crf_viterbi_accuracy: 0.9411 - val_loss: 27.4453 - val_crf_viterbi_accuracy: 0.9461
Epoch 4/15
12636/12636 [==============================] - 77s 6ms/step - loss: 32.4186 - crf_viterbi_accuracy: 0.9564 - val_loss: 27.4122 - val_crf_viterbi_accuracy: 0.9549
Epoch 5/15
12636/12636 [==============================] - 77s 6ms/step - loss: 32.3873 - crf_viterbi_accuracy: 0.9640 - val_loss: 27.3963 - val_crf_viterbi_accuracy: 0.9595
Epoch 6/15
12636/12636 [==============================] - 77s 6ms/step - loss: 32.3679

## Predict on Training Set

In [0]:
pred = model_2.predict(X, verbose=1)

14041/14041 [==============================] - 193s 14ms/step


In [0]:
# TRain Eval
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted')
print(score)

              precision    recall  f1-score   support

         LOC       0.97      0.96      0.97      8297
        MISC       0.95      0.88      0.91      4593
           O       0.99      1.00      0.99    169578
         ORG       0.97      0.89      0.93     10025
         PAD       1.00      1.00      1.00   1537463
         PER       0.99      0.98      0.99     11128

    accuracy                           1.00   1741084
   macro avg       0.98      0.95      0.97   1741084
weighted avg       1.00      1.00      1.00   1741084

0.9985804487779998


In [0]:
report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)
#F1 Score
score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)

              precision    recall  f1-score   support

        MISC       0.95      0.88      0.91      4593
         LOC       0.97      0.96      0.97      8297
         ORG       0.97      0.89      0.93     10025
         PER       0.99      0.98      0.99     11128

   micro avg       0.98      0.94      0.96     34043
   macro avg       0.97      0.93      0.95     34043
weighted avg       0.98      0.94      0.96     34043

0.9550323720744915


## Predict on Test Set

In [0]:
# Convert each sentence from list of Token to list of word_index
X_test = [[word2idx[w[0].lower()] for w in s] for s in test]

# Padding each sentence to have the same lenght
X_test = pad_sequences(maxlen=MAX_LEN, sequences=X_test, padding="post", value=word2idx["PAD"])
X_test

# Convert Tag/Label to tag_index
y_test = [[tag2idx[get_labeler((w[1]))] for w in s] for s in test]

# Padding each sentence to have the same lenght
y_test = pad_sequences(maxlen=MAX_LEN, sequences=y_test, padding="post", value=tag2idx["PAD"])

# One-Hot encode categorical labels
y_test = [to_categorical(i, num_classes=n_tags+1) for i in y_test]

In [0]:
pred = model_2.predict(X_test, verbose=1)

3453/3453 [==============================] - 47s 14ms/step


In [0]:
#pred_cat = model.predict(X_tr)
preds = np.argmax(pred, axis=-1)
y_tr_true = np.argmax(y_test, -1)

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in preds]
y_tr_true_tag = [[idx2tag[i] for i in row] for row in y_tr_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=y_tr_true_tag,labels=setlabel)
print(report)

score=flat_f1_score(y_pred=pred_tag, y_true=y_tr_true_tag,average='weighted',labels=setlabel)
print(score)


              precision    recall  f1-score   support

        MISC       0.71      0.70      0.70       918
         LOC       0.87      0.89      0.88      1925
         ORG       0.87      0.74      0.80      2496
         PER       0.96      0.94      0.95      2773

   micro avg       0.88      0.84      0.86      8112
   macro avg       0.85      0.81      0.83      8112
weighted avg       0.88      0.84      0.86      8112

0.8572302440182686
